
# Lab 9: Build a Log Aggregator

In this lab, you will create your own log generator, build a command-line utility that scans log files, summarizes their contents, and provides insight into system behavior. Data structures to track log message levels such as `INFO`, `WARNING`, `ERROR`, and `CRITICAL`.

This lab reinforces:
- File I/O
- Pattern recognition (regex)
- Dictionaries and counters
- Functions and modularity
- CLI arguments, logging



## Part 1: Create Log files (20%)
Using the the following example log format below create a **python file** that will log errors In a structured tree format 

You will find examples in the folder called Logs that you can use to build your program.

Remember set of logs should have a varied levels of log entries (`INFO`, `WARNING`, `ERROR`, `CRITICAL`) and tailored message types for different service components.
You must create 5 structured logs here are some examples:

    sqldb
    ui
    frontend.js
    backend.js
    frontend.flask
    backend.flask

You may use chat GPT to create sample outputs NOT THE LOGS. IE:

    System failure
    Database corruption
    Disk failure detected
    Database corruption


In [ ]:
import logging

# Create logger for file logging
logger = logging.getLogger("file_logger")
logger.setLevel(logging.INFO)

# Create file handler
file_handler = logging.FileHandler("example_log.log")
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

# Add handler to the logger
logger.addHandler(file_handler)

loggers = {
    "sqldb": logging.getLogger("sqldb"),
    "ui": logging.getLogger("ui"),
    "frontend.js": logging.getLogger("frontend.js"),
    "backend.js": logging.getLogger("backend.js"),
    "frontend.flask": logging.getLogger("frontend.flask"),
    "backend.flask": logging.getLogger("backend.flask"),
}

for log in loggers.values():
    log.addHandler(file_handler)
    log.setLevel(logging.INFO) 

def log_messages():
    loggers["sqldb"].info("user logged in with correct password")
    loggers["sqldb"].error("file not found")
    loggers["ui"].warning("disk space low")
    loggers["frontend.js"].critical("there is no more disk space")
    loggers["backend.flask"].error("file not found")

if __name__ == "__main__":
    log_messages()


### Example Log Format

You will work with logs that follow this simplified structure:

```
2025-04-11 23:20:36,913 | my_app | INFO | Request completed
2025-04-11 23:20:36,914 | my_app.utils | ERROR | Unhandled exception
2025-04-11 23:20:36,914 | my_app.utils.db | CRITICAL | Disk failure detected
```


## Part 2: Logging the Log File (40%)
    New File
### Part 2a: Read the Log File (see lab 7) (10%)


Write a function to read the contents of a log file into a list of lines. Handle file errors gracefully.

### Part 2b: Parse Log Lines (see code below if you get stuck) (10%)

Use a regular expression to extract:
- Timestamp
- Log name
- Log level
- Message

### Part 2c: Count Log Levels (20%)

Create a function to count how many times each log level appears. Store the results in a dictionary. Then output it as a Json File
You may pick your own format but here is an example. 
```python
{
    "INFO": 
    {
        "Request completed": 42, 
        "Heartbeat OK": 7
    }

    "WARNING":
    {
        ...
    }
}

```


In [ ]:
#Part 2a
def read_the_log_file(filename):
    log_lines = [] 
    try:
        with open(filename, 'r') as file: 
            log_lines = file.readlines()  # Read all lines into the list
        print("file read successfully")
    except FileNotFoundError:
        print("file not found")

    return log_lines

In [ ]:
#Part 2b
import re

def parse_log_lines(log_lines):
    # Define the regex pattern to match log entries
    regex_pattern = r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}) - (\w+) - (\w+) - (.*)'

    parsed_logs = [] 
    for line in log_lines:
        match = re.match(regex_pattern, line.strip())  # Match the line against the regex pattern
        if match:
            log_entry = {
                'timestamp': match.group(1),
                'log_name': match.group(2),
                'log_level': match.group(3),
                'message': match.group(4)
            }
            parsed_logs.append(log_entry)  # Add the parsed entry to the list
        else:
            print("Log entry does not match the expected format:", line)

    return parsed_logs

In [ ]:
#Part 2c
import json

def count_log_levels(parsed_logs):
    log_level_count = {}

    for log in parsed_logs:
        level = log['log_level']
        message = log['message']

        if level not in log_level_count:
            log_level_count[level] = {}

        if message not in log_level_count[level]:
            log_level_count[level][message] = 0

        log_level_count[level][message] += 1

    return log_level_count

def save_to_json(log_counts, filename='log_counts.json'):
    with open(filename, 'w') as json_file:
        json.dump(log_counts, json_file, indent=4)

if __name__ == "__main__":
    log_file_name = 'example_log.log' 
    parsed_log_entries = read_the_log_file(log_file_name)  

    log_counts = count_log_levels(parsed_log_entries)

    save_to_json(log_counts)


## Step 3: Generate Summary Report (40%)
    New File
### Step 3a (20%):
 Develop a function that continuously monitors your JSON file(s) and will print a real-time summary of log activity. It should keep count of the messages grouped by log level (INFO, WARNING, ERROR, CRITICAL) and display only the critical messages. (I.e. If new data comes in the summary will change and a new critical message will be printed)
 - note: do not reprocess the entire file on each update.  

### Step 3b: Use a Matplotlib (Lecture 10) (20%)
Develop a function that continuously monitors your JSON file(s) and will graph in real-time a bar or pie plot of each of the errors.  (a graph for each log level). 
- The graph should show the distribution of log messages by level  (INFO, WARNING, ERROR, CRITICAL)  


### Critical notes:
- Your code mus use Daemon Threads (Lecture 14)
- 3a and 3b do not need to run at the same time. 


In [ ]:
#Part 3a
import json
import os
import time
import threading

LOG_FILE = "log_counts.json" 
WAIT_TIME = 1 #Time between checks

# Function to load the JSON log summary
def load_logs(filepath):
    try:
        with open(filepath, "r") as f:
            return json.load(f)
    except Exception as e:
        print("Problem reading the log summary:", e)
        return {}

# Keeps track of new messages
previous_criticals = {}

# Main monitoring function
def watch_logs():
    last_updated = 0

    while True:
        if os.path.exists(LOG_FILE):
            current_time = os.path.getmtime(LOG_FILE)
            if current_time != last_updated:
                last_updated = current_time
                logs = load_logs(LOG_FILE)

                print("\nLog Summary")
                for level, msgs in logs.items():
                    total = sum(msgs.values())
                    print(f"{level}: {total} entries")

                    if level == "CRITICAL":
                        for msg, count in msgs.items():
                        
                            old_count = previous_criticals.get(msg, 0)
                            if count > old_count:
                                print(f"NEW CRITICAL: {msg} ({count})")
                                previous_criticals[msg] = count

        time.sleep(WAIT_TIME)

# Start thread
if __name__ == "__main__":
    t = threading.Thread(target=watch_logs)
    t.daemon = True
    t.start()

    print("Monitoring logs. Press Ctrl+C to stop\n")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\nStopped monitoring.")

In [ ]:
#Part 3b
import json
import os
import time
import threading
import matplotlib.pyplot as plt

LOG_FILE = "log_summary.json" 
WAIT_TIME = 1


def load_logs():
    try:
        with open(LOG_FILE, "r") as f:
            return json.load(f)
    except Exception as e:
        print("Couldn't read log file:", e)
        return {}

# Main function for plotting
def plot_logs():
    plt.ion()  
    fig, ax = plt.subplots()
    last_updated = 0

    while True:
        if os.path.exists(LOG_FILE):
            current_time = os.path.getmtime(LOG_FILE)
            if current_time != last_updated:
                last_updated = current_time
                logs = load_logs()

                # Reset chart
                ax.clear()

                levels = []
                totals = []

                for level, msgs in logs.items():
                    levels.append(level)
                    totals.append(sum(msgs.values()))

                ax.bar(levels, totals, color=['blue', 'orange', 'yellow', 'red'])
                ax.set_title("Log Level Distribution")
                ax.set_ylabel("Count")
                ax.set_xlabel("Log Level")

                plt.draw()
                plt.pause(0.1)

        time.sleep(WAIT_TIME)

# Start graphing in a daemon thread
if __name__ == "__main__":
    t = threading.Thread(target=plot_logs)
    t.daemon = True
    t.start()

    print("Graphing logs. Press Ctrl+C to stop\n")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\nStopped graphing.")